# TSA Chapter 10: ACF of Sunspot Numbers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10_sunspots_acf/TSA_ch10_sunspots_acf.ipynb)

This notebook demonstrates:
- Computing the ACF of yearly sunspot data to reveal the ~11-year solar cycle via peaks at lags 11 and 22.

In [ ]:
!pip install statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from matplotlib.patches import Patch
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Load sunspot data and compute ACF
data = sm.datasets.sunspots.load_pandas().data
data.index = pd.Index(data['YEAR'].astype(int))
data = data.loc[1900:2008]

acf_vals, confint = acf(data['SUNACTIVITY'], nlags=40, alpha=0.05)
lags = np.arange(len(acf_vals))

fig, ax = plt.subplots(figsize=(7, 3))
colors = [COLORS['red'] if l in [11, 22] else COLORS['blue'] for l in lags[1:]]
ax.bar(lags[1:], acf_vals[1:], width=0.7, color=colors)
ci_upper = confint[1:, 1] - acf_vals[1:]
ax.fill_between(lags[1:], -ci_upper, ci_upper, color=COLORS['blue'], alpha=0.12)
ax.axhline(0, color='black', lw=0.5)

for lag in [11, 22]:
    ax.annotate(f'lag {lag}', xy=(lag, acf_vals[lag]),
                xytext=(lag + 2, acf_vals[lag] + 0.08),
                arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=0.8),
                fontsize=8, color=COLORS['red'])

legend_elements = [Patch(facecolor=COLORS['blue'], label='ACF'),
                   Patch(facecolor=COLORS['red'], label='Peaks at lag 11, 22'),
                   Patch(facecolor=COLORS['blue'], alpha=0.12, label='95% CI')]
ax.legend(handles=legend_elements, loc='upper center',
          bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
ax.set_xlabel('Lag'); ax.set_ylabel('ACF')
ax.set_title('ACF of Sunspot Numbers')
fig.tight_layout()
save_chart(fig, 'sunspots_acf')
plt.show()